In [2]:
import pandas as pd
import json

In [3]:
json_file = "v1.0-2022-03-17-ror-data.json"
with open(json_file) as datafile:
  data = json.load(datafile)
dataframe = pd.DataFrame(data)

In [8]:
print(dataframe.columns)

Index(['id', 'name', 'types', 'links', 'aliases', 'acronyms', 'status',
       'wikipedia_url', 'labels', 'email_address', 'ip_addresses',
       'established', 'country', 'relationships', 'addresses', 'external_ids'],
      dtype='object')


In [14]:
print(dataframe[dataframe['name'] == 'Erasmus MC'])

                            id        name         types  \
260  https://ror.org/018906e22  Erasmus MC  [Healthcare]   

                          links                              aliases acronyms  \
260  [http://www.erasmusmc.nl/]  [Erasmus University Medical Center]       []   

     status                             wikipedia_url labels email_address  \
260  active  https://en.wikipedia.org/wiki/Erasmus_MC     []          None   

    ip_addresses  established  \
260           []       2011.0   

                                               country  \
260  {'country_code': 'NL', 'country_name': 'Nether...   

                                         relationships  \
260  [{'type': 'Related', 'label': 'Erasmus Univers...   

                                             addresses  \
260  [{'line': None, 'lat': 51.91, 'lng': 4.4675, '...   

                                          external_ids  
260  {'ISNI': {'preferred': None, 'all': ['00000004...  


In [24]:
df_temp = dataframe[['id','name']]
df_temp['id'] = df_temp['id'].str[16:]
print(df_temp)

               id                                              name
0       019wvm592                    Australian National University
1       02bfwt286                                 Monash University
2       00rqy9422                          University of Queensland
3       01sf06y89                              Macquarie University
4       03r8z3t63                                       UNSW Sydney
...           ...                                               ...
102554  04y2aw426                                         De Bastei
102555  005k57c98                       Udako Euskal Unibertsitatea
102556  03zmjc935                                 US Forest Service
102557  05cynqq24  Deutsche Gesellschaft für Erziehungswissenschaft
102558  01897f782                   Universidad Nacional de Juliaca

[102559 rows x 2 columns]


/tmp/ipykernel_32188/2356695637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['id'] = df_temp['id'].str[16:]
